In [2]:
import numpy as np
import pandas as pd

# **Ukraine Damages**

In [3]:
ukraine_damages = pd.read_csv('data/ukraine-damages.csv', sep='|')
ukraine_damages.sort_values('date_of_event', ascending=False).head(2)

,damage_id,iso3,country,gid_1,oblast,rayon,type_of_infrastructure,if_other_what,date_of_event,source_name,source_date,source_link,additional_sources,extent_of_damage,_internal_filter_date,_weights,access_subindicator,pcode
17550,D26179,UKR,Ukraine,['UKR.8_1'],Kharkivska,Iziumskyi,Industrial/Business/Enterprise facilities,NaN,2025-10-01,UNN,2025-10-01,https://unn.ua/en/news/russian-missile-strike-...,NaN,Partially damaged,2025-10-01,0.7,['7.2'],UA63
17562,D26190,UKR,Ukraine,['UKR.8_1'],Kharkivska,Iziumskyi,Industrial/Business/Enterprise facilities,NaN,2025-10-01,UNN,2025-10-02,https://unn.ua/en/news/russian-missile-strike-...,NaN,Partially damaged,2025-10-01,0.7,['7.2'],UA63


# **Ukraine Events**

### columns/head

In [4]:
ukraine_events = pd.read_csv('data/ukraine-events.csv')

# add columns
ukraine_events["aggr_is_military"] = ukraine_events['actor1'].str.startswith('Military')
ukraine_events["def_is_military"] = ukraine_events['actor2'].str.startswith('Military')

ukraine_events['aggr_military'] = np.select(
    condlist = [
        ukraine_events['actor1'].str.contains('Ukraine', case=False, na=False) &
        ukraine_events['actor1'].str.contains('military', case=False, na=False),

        ukraine_events['actor1'].str.contains('Russia', case=False, na=False) &
        ukraine_events['actor1'].str.contains('military', case=False, na=False)
    ], choicelist = [
        'Ukraine',
        'Russia'
    ], default = None
)
ukraine_events['def_military'] = np.select(
    condlist = [
        ukraine_events['actor2'].str.contains('Ukraine', case=False, na=False) &
        ukraine_events['actor2'].str.contains('military', case=False, na=False),

        ukraine_events['actor2'].str.contains('Russia', case=False, na=False) &
        ukraine_events['actor2'].str.contains('military', case=False, na=False)
    ], choicelist = [
        'Ukraine',
        'Russia'
    ], default = None
)
ukraine_events['military_interaction'] = ukraine_events['aggr_military'].astype(str) + '-' + ukraine_events['def_military'].astype(str)

print(*list(ukraine_events.columns), sep='\n')
ukraine_events.head(2)

event_id_cnty
event_date
year
time_precision
disorder_type
event_type
sub_event_type
actor1
assoc_actor_1
inter1
actor2
assoc_actor_2
inter2
interaction
civilian_targeting
iso
region
country
admin1
admin2
admin3
location
latitude
longitude
geo_precision
source
source_scale
notes
fatalities
tags
timestamp
population_1km
population_2km
population_5km
population_best
aggr_is_military
def_is_military
aggr_military
def_military
military_interaction


,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,timestamp,population_1km,population_2km,population_5km,population_best,aggr_is_military,def_is_military,aggr_military,def_military,military_interaction
0,UKR31094,2020-01-01,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Ukraine),Svoboda,Protesters,...,1618585723,6291.0,37747.0,206833.0,6291.0,False,NaN,None,None,None-None
1,UKR31107,2020-01-01,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Ukraine),Svoboda; Traditions and Order,Protesters,...,1618585724,5307.0,34769.0,151764.0,5307.0,False,NaN,None,None,None-None


In [5]:
ukraine_events['event_id_cnty'].is_unique

True

### interaction

In [6]:
ukraine_events['interaction'].unique()

array(['Protesters only', 'Rebel group only', 'State forces-Rebel group',
       'State forces only', 'State forces-Protesters',
       'State forces-External/Other forces', 'Rioters only',
       'State forces-Civilians', 'State forces-Political militia',
       'Political militia-Civilians', 'Rioters-Civilians',
       'Protesters-External/Other forces',
       'External/Other forces-Civilians', 'Political militia only',
       'Rebel group-Civilians', 'Political militia-Political militia',
       'Rioters-Protesters', 'Rioters-Rioters', 'State forces-Rioters',
       'Protesters-Protesters', 'Rioters-External/Other forces',
       'State forces-State forces',
       'Political militia-External/Other forces',
       'Rebel group-Political militia', 'Civilians only',
       'Political militia-Protesters', 'External/Other forces only',
       'Rebel group-Rebel group',
       'Identity militia-External/Other forces',
       'Identity militia-Civilians',
       'External/Other forces-Ex

### location

In [7]:
ukraine_events["country"].value_counts()

country
Ukraine    164792
Russia      21546
Name: count, dtype: int64

In [8]:
ukraine_events.query("inter1 == 'State forces'")['country'].value_counts()

country
Ukraine    31098
Russia      3633
Name: count, dtype: int64

### disorder type and inter1 exploration

In [9]:
actors = ukraine_events['actor1'].unique()
state_actors = ukraine_events.query("inter1 == 'State forces'")['actor1'].unique()
print(*sorted(actors), sep='\n')

Africa Corps
Atesh
BKH: We Are Common People
Berdiansk Communal Militia (Ukraine)
Bezimenne Communal Militia (Ukraine)
Bilovodsk Communal Militia (Ukraine)
Black Bridge
Bucha Communal Militia (Ukraine)
Chornobaivka Communal Militia (Ukraine)
Civilians (International)
Civilians (Russia)
Civilians (Ukraine)
Donetsk Communal Militia (Ukraine)
Donetsk People's Republic - Police
Dzhankoy Communal Militia (Ukraine)
ER: United Russia
Enerhodar Communal Militia (Ukraine)
Erzya Resistance
Frunze Communal Militia (Ukraine)
Government of Poland (2015-2023)
Government of Russia (2000-)
Government of Ukraine (2019-)
Henichesk Communal Militia (Ukraine)
Hola Prystan Communal Militia (Ukraine)
Islamic State Caucasus Province (ISCP)
Islamic State Khorasan Province (ISKP)
Islamist Militia (Russia)
Kakhovka Communal Militia (Ukraine)
Kamianka-Dniprovska Communal Militia (Ukraine)
Kherson Communal Militia (Ukraine)
Koktebel Communal Militia (Ukraine)
Kostohryzove Communal Militia (Ukraine)
LSR: Freedom o

In [10]:
temp = ukraine_events.query('disorder_type == "Political violence" and inter1 == "State forces" and actor1.str.contains("Military")')[['actor1','inter1']].value_counts()
temp_df = temp.reset_index(name='count')
print(temp_df.to_string())

                                                                            actor1        inter1  count
0                                               Military Forces of Ukraine (2019-)  State forces  26097
1                                     Military Forces of Ukraine (2019-) Air Force  State forces   1321
2                                      Military Forces of Russia (2000-) Air Force  State forces    625
3                                                Military Forces of Russia (2000-)  State forces    457
4                                       Military Forces of Ukraine (2019-) Marines  State forces    451
5                                          Military Forces of Ukraine (2019-) Navy  State forces     32
6                                Military Forces of Ukraine (2019-) Special Forces  State forces     23
7                                Military Forces of Ukraine (2019-) National Guard  State forces     18
8                                        Military Forces of Russ

In [11]:
state_forces = ukraine_events.query('disorder_type == "Political violence" and inter1 == "State forces"')

In [12]:
temp = ukraine_events.query('inter1 == "State forces"')[['actor1','inter1']].value_counts()
temp_df = temp.reset_index(name='count')
print(temp_df.to_string())

                                                                            actor1        inter1  count
0                                               Military Forces of Ukraine (2019-)  State forces  28274
1                                                Military Forces of Russia (2000-)  State forces   2246
2                                     Military Forces of Ukraine (2019-) Air Force  State forces   1351
3                                      Military Forces of Russia (2000-) Air Force  State forces    644
4              Police Forces of Ukraine (2019-) State Emergency Service of Ukraine  State forces    497
5                                                  Police Forces of Russia (2000-)  State forces    469
6                                       Military Forces of Ukraine (2019-) Marines  State forces    455
7                                                    Government of Ukraine (2019-)  State forces     87
8                     Police Forces of Ukraine (2019-) Security 

In [13]:
actor_inter_combos = ukraine_events[['actor1','inter1']].value_counts().reset_index().sort_values('actor1')
for row in actor_inter_combos.iterrows():
    print('"', row[1][0], '", "', row[1][1], '"', sep='')

"Africa Corps", "External/Other forces"
"Atesh", "Political militia"
"BKH: We Are Common People", "Political militia"
"Berdiansk Communal Militia (Ukraine)", "Identity militia"
"Bezimenne Communal Militia (Ukraine)", "Identity militia"
"Bilovodsk Communal Militia (Ukraine)", "Identity militia"
"Black Bridge", "Political militia"
"Bucha Communal Militia (Ukraine)", "Identity militia"
"Chornobaivka Communal Militia (Ukraine)", "Identity militia"
"Civilians (International)", "Civilians"
"Civilians (Russia)", "Civilians"
"Civilians (Ukraine)", "Civilians"
"Donetsk Communal Militia (Ukraine)", "Identity militia"
"Donetsk People's Republic - Police", "Rebel group"
"Dzhankoy Communal Militia (Ukraine)", "Identity militia"
"ER: United Russia", "Political militia"
"Enerhodar Communal Militia (Ukraine)", "Identity militia"
"Erzya Resistance", "Political militia"
"Frunze Communal Militia (Ukraine)", "Identity militia"
"Government of Poland (2015-2023)", "External/Other forces"
"Government of Russ

C:\Users\seths\AppData\Local\Temp\ipykernel_25472\2610319769.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('"', row[1][0], '", "', row[1][1], '"', sep='')


In [14]:
for row in actor_inter_combos[actor_inter_combos.duplicated(subset=['actor1'], keep=False)].iterrows():
    print('"', row[1][0], '", "', row[1][1], '"', sep='')

"Government of Russia (2000-)", "State forces"
"Government of Russia (2000-)", "External/Other forces"
"Military Forces of Russia (2000-)", "External/Other forces"
"Military Forces of Russia (2000-)", "State forces"
"Military Forces of Russia (2000-) Air Force", "State forces"
"Military Forces of Russia (2000-) Air Force", "External/Other forces"
"Military Forces of Russia (2000-) Chechen Battalion of Ramzan Kadyrov", "External/Other forces"
"Military Forces of Russia (2000-) Chechen Battalion of Ramzan Kadyrov", "State forces"
"Military Forces of Russia (2000-) National Guard", "External/Other forces"
"Military Forces of Russia (2000-) National Guard", "State forces"
"Military Forces of Russia (2000-) Navy", "State forces"
"Military Forces of Russia (2000-) Navy", "External/Other forces"
"Military Forces of Russia (2000-) Special Forces", "State forces"
"Military Forces of Russia (2000-) Special Forces", "External/Other forces"
"Military Forces of Ukraine (2019-)", "State forces"
"Mil

C:\Users\seths\AppData\Local\Temp\ipykernel_25472\1710568188.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('"', row[1][0], '", "', row[1][1], '"', sep='')


In [15]:
ukraine_events.query('actor1 == "Military Forces of Russia (2000-)"')['assoc_actor_1'].value_counts()

assoc_actor_1
Military Forces of Russia (2000-) Air Force                                                                                                                                                                                    4221
Wagner Group                                                                                                                                                                                                                    180
Donetsk People's Militia; NAF: United Armed Forces of Novorossiya                                                                                                                                                                43
Russian Occupation Government (2022-)                                                                                                                                                                                            20
Military Forces of Russia (2000-) 1st Donetsk Army Corps                  

In [16]:
temp = ukraine_events.query('actor1.str.startswith("Military")')[['actor1','inter1']].value_counts()
temp_df = temp.reset_index(name='count')
temp_df['Country'] = temp_df["actor1"].case_when([
    (temp_df.eval("actor1.str.contains('Russia')"), "Russia"),
    (temp_df.eval("actor1.str.contains('Ukraine')"), "Ukraine"),
    (temp_df.eval("actor1.str.contains('Belarus')"), "Belarus")
])
print(temp_df.to_string())

                                                                            actor1                 inter1  count  Country
0                                                Military Forces of Russia (2000-)  External/Other forces  99409   Russia
1                                               Military Forces of Ukraine (2019-)           State forces  28274  Ukraine
2                                      Military Forces of Russia (2000-) Air Force  External/Other forces  20109   Russia
3                                     Military Forces of Ukraine (2019-) Air Force  External/Other forces   6493  Ukraine
4                                               Military Forces of Ukraine (2019-)  External/Other forces   5381  Ukraine
5                                                Military Forces of Russia (2000-)           State forces   2246   Russia
6                                     Military Forces of Ukraine (2019-) Air Force           State forces   1351  Ukraine
7                       

In [17]:
ukraine_events[['aggr_military', 'def_military']].value_counts(dropna=False).reset_index()
pd.crosstab(ukraine_events['aggr_military'], ukraine_events['def_military'], dropna=False)

def_military,Russia,Ukraine,NaN
aggr_military,,,
Russia,48,46348,76170
Ukraine,12165,14,29932
NaN,650,6351,14660


In [18]:
temp = ukraine_events.query('disorder_type == "Political violence"')
pd.crosstab(temp['aggr_military'], temp['def_military'], dropna=False)

def_military,Russia,Ukraine,NaN
aggr_military,,,
Russia,36,40089,75487
Ukraine,9965,12,29853
NaN,179,6163,4579


In [19]:
ukraine_events['military_interaction'].value_counts()

military_interaction
Russia-None        76170
Russia-Ukraine     46348
Ukraine-None       29932
None-None          14660
Ukraine-Russia     12165
None-Ukraine        6351
None-Russia          650
Russia-Russia         48
Ukraine-Ukraine       14
Name: count, dtype: int64

In [20]:
temp = ukraine_events.query('aggr_military == "Russia"')
print(temp.value_counts('disorder_type').reset_index())
temp = ukraine_events.query('aggr_military == "Ukraine"')
print(temp.value_counts('disorder_type').reset_index())

            disorder_type   count
0      Political violence  115612
1  Strategic developments    6954
            disorder_type  count
0      Political violence  39830
1  Strategic developments   2281


In [21]:
for r in ukraine_events.query("aggr_military == 'Russia' and def_military.isnull()").drop(
    columns=['event_id_cnty', 'year', 'time_precision', 'population_1km', 'population_5km', 'population_best', 'source_scale', 'source', 'geo_precision', 'latitude', 'longitude']
).head(5).itertuples():
    print()
    print(*list(r), sep='\n')
    print()


6488
2020-05-22
Political violence
Battles
Armed clash
Military Forces of Russia (2000-) National Guard
nan
State forces
Islamic State Caucasus Province (ISCP)
nan
Rebel group
State forces-Rebel group
nan
643
Europe
Russia
Republic of Dagestan
Khasavyurtovskiy
nan
Goksuv
On 22 May 2020, Russian National Guard forces engaged in an armed clash with alleged ISIS militants near the village of Goksuv, killing 6, with one National Guard officer suffering serious injuries.
6
nan
1753971887
5561.0
True
False
Russia
None
Russia-None


13644
2021-03-15
Political violence
Violence against civilians
Attack
Military Forces of Russia (2000-) National Guard - Special Purpose Police Unit
nan
State forces
Civilians (Russia)
nan
Civilians
State forces-Civilians
Civilian targeting
643
Europe
Russia
Khabarovsk
Khabarovsk
nan
Khabarovsk
Around 15 March 2021 (month of), Special Purpose Police Unit (OMON) officers allegedly physically assaulted and beat a political activist while conducting a raid at his ho

In [22]:
temp = ukraine_events.query('aggr_is_military == True')
pd.crosstab(temp['disorder_type'], temp['event_type'], dropna=False)

event_type,Battles,Explosions/Remote violence,Strategic developments,Violence against civilians
disorder_type,,,,
Political violence,34043,120268,0,1131
Strategic developments,0,0,9237,0


In [23]:
temp = ukraine_events.query('aggr_is_military == True')
pd.crosstab(temp['event_type'], temp['sub_event_type'], dropna=False)

sub_event_type,Abduction/forced disappearance,Agreement,Air/drone strike,Armed clash,Arrests,Attack,Change to group/activity,Chemical weapon,Disrupted weapons use,Government regains territory,Grenade,Headquarters or base established,Looting/property destruction,Non-state actor overtakes territory,Non-violent transfer of territory,Other,Remote explosive/landmine/IED,Sexual violence,Shelling/artillery/missile attack,Suicide bomb
event_type,,,,,,,,,,,,,,,,,,,,
Battles,0,0,0,33257,0,0,0,0,0,369,0,0,0,417,0,0,0,0,0,0
Explosions/Remote violence,0,0,28445,0,0,0,0,4,0,0,5,0,0,0,0,0,439,0,91374,1
Strategic developments,0,1,0,0,5,0,181,0,8560,0,0,12,271,0,145,62,0,0,0,0
Violence against civilians,409,0,0,0,0,625,0,0,0,0,0,0,0,0,0,0,0,97,0,0


In [24]:
temp = ukraine_events.query('aggr_is_military == True & sub_event_type in ["Armed clash", "Air/drone strike", "Shelling/artillery/missile attack", "Disrupted weapons use"]')
temp['sub_event_type'].value_counts()

sub_event_type
Shelling/artillery/missile attack    91374
Armed clash                          33257
Air/drone strike                     28445
Disrupted weapons use                 8560
Name: count, dtype: int64

In [25]:
temp = ukraine_events.query('aggr_is_military == True & sub_event_type in ["Armed clash", "Air/drone strike", "Shelling/artillery/missile attack", "Disrupted weapons use"]')
pd.crosstab(temp['military_interaction'], temp['sub_event_type'], dropna=False).reindex(['Russia-Ukraine', 'Ukraine-Russia', 'Russia-None', 'Ukraine-None', 'Russia-Russia', 'Ukraine-Ukraine'], axis=0)

sub_event_type,Air/drone strike,Armed clash,Disrupted weapons use,Shelling/artillery/missile attack
military_interaction,,,,
Russia-Ukraine,4233,25182,6245,10238
Ukraine-Russia,1018,4707,2009,3842
Russia-None,16336,24,245,57707
Ukraine-None,6853,3318,51,19574
Russia-Russia,3,22,8,9
Ukraine-Ukraine,2,4,2,4


### location

In [44]:
temp = ukraine_events.query('aggr_is_military == True & sub_event_type in ["Armed clash", "Air/drone strike", "Shelling/artillery/missile attack", "Disrupted weapons use"]')
temp[['country','admin1']].value_counts().head(40)

country  admin1         
Ukraine  Donetsk            60971
         Kharkiv            19080
         Zaporizhia         15146
         Kherson            14392
         Sumy               13781
Russia   Belgorod           10349
Ukraine  Luhansk             9729
         Chernihiv           4039
         Mykolaiv            3370
         Dnipropetrovsk      3301
Russia   Kursk               3149
Ukraine  Odesa                702
         Kyiv                 452
         Crimea               426
Russia   Bryansk              371
Ukraine  Kyiv City            369
         Poltava              191
         Kirovohrad           186
         Khmelnytskyi         164
         Zhytomyr             154
         Cherkasy             149
         Vinnytsia            122
Russia   Rostov               109
Ukraine  Lviv                 101
Russia   Voronezh              93
         Krasnodar             91
         Moscow Oblast         61
         Oryol                 54
         Kaluga        